# Trabajo Práctico 6
### Dante Saavedra

### Nota: 
Descargar Fichero paises.json 

#### Importar Libreria

In [ ]:
from neo4jrestclient.client import GraphDatabase
from neo4jrestclient import client
import pandas as pd

### Conectarse a la neo4j

In [ ]:
db = GraphDatabase("http://localhost:7474", username="neo4j", password="root")

### Cargar Datos a Neo4j

#### Lectura de fichero json con la información

In [ ]:
datos = pd.read_json("paises.json")

#### Cargar los continentes

In [ ]:
continentes = datos['continent'].unique().tolist()
continentesGraph = []
continentesNeo = db.labels.create("Continente")
for c in range(len(continentes)):
    continentesGraph.append(db.nodes.create(name=continentes[c]))
    continentesNeo.add(continentesGraph[c])

#### Cargar los paises y crear relación a su continente

In [ ]:
paisesGraph = []
paisesNeo = db.labels.create("Pais")
for p in range(len(datos)):
    paisesGraph.append(db.nodes.create(name=datos.iloc[p].country))
    paisesNeo.add(paisesGraph[p])
    iContinent = continentes.index(datos.iloc[p].continent)
    paisesGraph[p].relationships.create("Pertenece", continentesGraph[iContinent])

### Realizar consulta de que paises y sus relaciones

In [ ]:
q = 'MATCH (p:Pais)-[r:Pertenece]->(c:Continente) RETURN p,type(r),c'
results = db.query(q, returns=(client.Node, str, client.Node))
for r in results:
    print("%s ---> %s ---> %s" % (r[0]["name"], r[1], r[2]["name"]))